# அமர்வு 1 – Foundry Local (சாட் பூட்ஸ்ட்ராப்)

இந்த நோட்புக் Foundry Local-ஐ பூட்ஸ்ட்ராப் செய்கிறது, விருப்பமான மாடல் அலியாஸ்-ஐ பதிவிறக்குகிறது, மேலும் ஒரு சாதாரண மற்றும் ஸ்ட்ரீமிங் சாட் நிறைவு செயல்படுத்துகிறது.


# சூழல்
இந்த அமர்வு ஒரு உள்ளூர் சிறிய மொழி மாதிரியை Foundry Local மூலம் பதிலளிக்கச் செய்ய தேவையான குறைந்தபட்சத்தை அறிமுகப்படுத்துகிறது. நீங்கள் செய்ய வேண்டியது:
- SDK / கிளையன்ட் சார்புகளை நிறுவவும்.
- தேர்ந்தெடுக்கப்பட்ட பெயருக்கு Foundry Local மேலாளரை ஆரம்பிக்கவும் (இயல்புநிலை: `phi-4-mini`).
- மாதிரி மெட்டாடேட்டாவில் விருப்ப புலங்களை ஏற்க ஒரு பாதுகாப்பான மங்கிக்‑பேட்ச் பயன்படுத்தவும்.
- ஒரு நிலையான உரையாடல் முடிவு கோரிக்கையை அனுப்பவும்.
- பதில்களை டோக்கன்‑டோக்கனாக ஸ்ட்ரீம் செய்யவும்.

இலக்கு உங்கள் உள்ளூர் ரன்டைம் மற்றும் நெட்வொர்க் பாதையை RAG, ரவுடிங் அல்லது ஏஜென்ட்களுக்கு மாறுவதற்கு முன் சரிபார்க்க வேண்டும்.


### விளக்கம்: சார்பு நிறுவல்
இந்த குறைந்தபட்ச உரையாடல் ஓட்டத்திற்குத் தேவையான Python தொகுப்புகளை நிறுவுகிறது:
- `foundry-local-sdk`: உள்ளூர் மாதிரிகள் மற்றும் சேவை வாழ்க்கைச்சுழற்சியை நிர்வகிக்க.
- `openai`: உரையாடல் முடிவுகளுக்கான பரிச்சயமான கிளையன்ட் சுருக்கம்.
- `rich`: நோட்புக் வெளியீட்டிற்கான அழகான அச்சிடல்.

மீண்டும் இயக்குவது பாதுகாப்பானது (idempotent). உங்கள் சூழல் ஏற்கனவே இதை கொண்டிருந்தால் தவிர்க்கவும்.


In [1]:
# Install required libraries (idempotent)
%pip install -q foundry-local-sdk openai rich


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### விளக்கம்: முக்கிய இறக்குமைகள்
நாட்குறிப்பில் முழுவதும் பயன்படுத்தப்படும் தொகுதிகளை கொண்டு வருகிறது:
- `FoundryLocalManager` உள்ளூர் மாடல் ரன்டைமைன் தொடர்பு கொள்ள.
- `OpenAI` கிளையண்ட், பரிச்சயமான சாட் கம்ப்ளீஷன் API மேற்பரப்பை மீண்டும் பயன்படுத்துவதற்காக.
- `rich.print` அழகான வெளியீட்டிற்காக.

இங்கே எந்த நெட்வொர்க் அழைப்புகளும் நடக்காது—இது பெயரிடப்பகுதியை தயாரிக்கிறது.


In [2]:
import os
from foundry_local import FoundryLocalManager
from foundry_local.models import FoundryModelInfo
from openai import OpenAI
from rich import print

### விளக்கம்: மேலாளர் தொடக்கம் & மெட்டாடேட்டா திருத்தம்
தேர்ந்தெடுக்கப்பட்ட அலியாஸ் க்கான `FoundryLocalManager` ஐ தொடங்குகிறது மற்றும் `promptTemplate` `null` ஆக இருக்கக்கூடிய சேவை பதில்களை நயமுடன் கையாள ஒரு பாதுகாப்பான மங்கீ-பேட்ச் பயன்படுத்துகிறது.

முக்கிய முடிவுகள்:
- சேவை நிலை மற்றும் முடுக்கத்தை உறுதிப்படுத்துகிறது.
- காசேடு செய்யப்பட்ட மாதிரிகளை பட்டியலிடுகிறது (உள்ளூர் சேமிப்பகத்தை சரிபார்க்கிறது).
- அலியாஸ் க்கான உறுதியான மாதிரி ஐடியை தீர்மானிக்கிறது (பின்னர் உரையாடல் அழைப்புகளில் பயன்படுத்தப்படுகிறது).

உங்கள் சேவை மெட்டாடேட்டாவில் சரிபார்ப்பு பிரச்சினைகளை சந்தித்தால், SDK ஐ பிரிக்காமல் சுத்தம் செய்ய இந்த முறை எப்படி உதவுகிறது என்பதை இது காட்டுகிறது.


In [3]:
# Monkeypatch to tolerate service responses where promptTemplate is null
_original_from_list_response = FoundryModelInfo.from_list_response

def _safe_from_list_response(response):  # type: ignore
    try:
        if isinstance(response, dict) and response.get("promptTemplate") is None:
            # Normalize to empty dict so pydantic validation passes
            response["promptTemplate"] = {}
    except Exception as e:  # pragma: no cover
        print(f"[yellow]Warning: safe wrapper encountered issue normalizing promptTemplate: {e}[/yellow]")
    return _original_from_list_response(response)

# Apply patch only once
if getattr(FoundryModelInfo.from_list_response, "__name__", "") != "_safe_from_list_response":
    FoundryModelInfo.from_list_response = staticmethod(_safe_from_list_response)  # type: ignore

ALIAS = os.getenv('FOUNDRY_LOCAL_ALIAS', 'phi-4-mini')
manager = FoundryLocalManager(ALIAS)
print(f'[bold green]Service running:[/bold green] {manager.is_service_running()}')
print(f'Endpoint: {manager.endpoint}')
print('Cached models:', manager.list_cached_models())
model_id = manager.get_model_info(ALIAS).id
print(f'Using model id: {model_id}')

Service running: True

Endpoint: http://127.0.0.1:50262/v1

Cached models:
[
    FoundryModelInfo(
        alias='phi-4-mini',
        id='Phi-4-mini-instruct-generic-gpu:4',
        version='4',
        execution_provider='WebGpuExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/Phi-4-mini-instruct-generic-gpu/versions/4',
        file_size_mb=3809,
        prompt_template={
            'system': '<|system|>{Content}<|end|>',
            'user': '<|user|>{Content}<|end|>',
            'assistant': '<|assistant|>{Content}<|end|>',
            'prompt': '<|user|>{Content}<|end|><|assistant|>'
        },
        provider='AzureFoundry',
        publisher='Microsoft',
        license='MIT',
        task='chat-completion',
        ep_override=None
    ),
    FoundryModelInfo(
        alias='qwen2.5-0.5b',
        id='qwen2.5-0.5b-instruct-generic-gpu:3',
        version='3',
        execution_provider='WebGpuExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/qwen2.5-0.5b-instruct-generic-gpu/versions/3',
        file_size_mb=700,
        prompt_template={
            'system': '<|im_start|>system\n{Content}<|im_end|>',
            'user': '<|im_start|>user\n{Content}<|im_end|>',
            'assistant': '<|im_start|>assistant\n{Content}<|im_end|>',
            'prompt': '<|im_start|>user\n{Content}<|im_end|>\n<|im_start|>assistant'
        },
        provider='AzureFoundry',
        publisher='Microsoft',
        license='apache-2.0',
        task='chat-completion',
        ep_override=None
    ),
    FoundryModelInfo(
        alias='phi-3.5-mini',
        id='Phi-3.5-mini-instruct-generic-gpu:1',
        version='1',
        execution_provider='WebGpuExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/Phi-3.5-mini-instruct-generic-gpu/versions/1',
        file_size_mb=2211,
        prompt_template={
            'prompt': '<|user|>\n{Content}<|end|>\n<|assistant|>',
            'assistant': '<|assistant|>\n{Content}<|end|>'
        },
        provider='AzureFoundry',
        publisher='Microsoft',
        license='MIT',
        task='chat-completion',
        ep_override=None
    )
]

Using model id: Phi-4-mini-instruct-generic-gpu:4

### விளக்கம்: அடிப்படை உரையாடல் நிறைவு
உள்ளூர் Foundry இறுதிப்புள்ளியில் `OpenAI`-இன் இணக்கமான கிளையண்டை உருவாக்கி, ஒரு ஒற்றை non-streaming உரையாடல் நிறைவைச் செய்கிறது. இங்கு கவனம் செலுத்தவும்:
- மாடல் பிழையின்றி பதிலளிக்க வேண்டும்.
- தாமதம் / வெளியீட்டு வடிவத்தை சரிபார்க்கவும்.
- வளங்களை பாதுகாக்க `max_tokens` அளவைக் குறைவாக வைத்திருக்கவும்.

இது தோல்வியடையுமானால், Foundry Local சேவை இயங்குகிறதா மற்றும் alias சரியாக தீர்க்கப்படுகிறதா என்பதை மீண்டும் சரிபார்க்கவும்.


In [4]:
client = OpenAI(base_url=manager.endpoint, api_key=manager.api_key or 'not-needed')
prompt = 'List two benefits of local inference for privacy.'
resp = client.chat.completions.create(
    model=model_id,
    messages=[{'role':'user','content':prompt}],
    max_tokens=120,
    temperature=0.5
)
print(resp.choices[0].message.content)

Local inference for privacy refers to the practice of performing data analysis on a local device without sending 
sensitive information to a central server. Two benefits of this approach are:


1. **Enhanced Privacy**: Local inference keeps personal data on the user's device, reducing the risk of data 
breaches and unauthorized access. Since the data is not transmitted over the network, it is less susceptible to 
interception by malicious actors.


2. **Data Sovereignty**: Users retain control over their data, as it does not leave their device. This means that 
individuals or organizations can comply with local data protection regulations, such as the General

### விளக்கம்: ஸ்ட்ரீமிங் சந்தாதி நிறைவு
சந்தாதி UX மற்றும் குறைந்த தாமதத்திற்காக டோக்கன் ஸ்ட்ரீமிங்கை விளக்குகிறது. வரிசையில் வரும் டெல்டாக்களை அடுத்தடுத்து அச்சிடுகிறது:
- ஆரம்பகால பகுதி வெளியீடு முக்கியமானது எனும் சந்தாதி UIகளுக்கு பயனுள்ளதாக உள்ளது.
- முழு நிறைவு தாமதத்துடன் ஒப்பிடும்போது டோக்கன் துரிதத்தை அளவிட அனுமதிக்கிறது.

இந்த முறைமையை டோக்கன்களை சேர்க்க, முன்னேற்றம் விட்ஜெட்டை புதுப்பிக்க, அல்லது நடுவில் உருவாக்கத்தை நிறுத்த மாற்றலாம்.


In [5]:
# Streaming example
stream = client.chat.completions.create(
    model=model_id,
    messages=[{'role':'user','content':'Give a one-sentence definition of edge AI.'}],
    stream=True,
    max_tokens=60,
    temperature=0.4
)
for chunk in stream:
    delta = chunk.choices[0].delta
    if delta and delta.content:
        print(delta.content, end='', flush=True)
print()

Edge

AI

refers

to

artificial

...

latency

and

bandwidth

usage

.

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**குறிப்பு**:  
இந்த ஆவணம் AI மொழிபெயர்ப்பு சேவை [Co-op Translator](https://github.com/Azure/co-op-translator) பயன்படுத்தி மொழிபெயர்க்கப்பட்டுள்ளது. நாங்கள் துல்லியத்திற்காக முயற்சிக்கிறோம், ஆனால் தானியங்கி மொழிபெயர்ப்புகளில் பிழைகள் அல்லது தவறுகள் இருக்கக்கூடும் என்பதை கவனத்தில் கொள்ளவும். அதன் தாய்மொழியில் உள்ள மூல ஆவணம் அதிகாரப்பூர்வ ஆதாரமாக கருதப்பட வேண்டும். முக்கியமான தகவல்களுக்கு, தொழில்முறை மனித மொழிபெயர்ப்பு பரிந்துரைக்கப்படுகிறது. இந்த மொழிபெயர்ப்பைப் பயன்படுத்துவதால் ஏற்படும் எந்த தவறான புரிதல்கள் அல்லது தவறான விளக்கங்களுக்கு நாங்கள் பொறுப்பல்ல.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
